# handle product sale detail

In [1]:
# read csv product sale detail
import pandas as pd
df_PrSlDl = pd.read_csv(r"D:\PURCHASING\test_NA\01. Product Sale Detail.csv",
                     parse_dates=["SDate","DocumentDate"],dayfirst=True,
                     skiprows=3,header=0,encoding='utf-8-sig')
df_PrSlDl.drop(["Textbox46","Textbox18","Textbox47","Textbox48"],axis=1,inplace=True)

# convert string to float
decimal_list = ["Amount","AmountReturn","Discount","DiscountAmount",
                "CashDiscount","Amount1","AmountReturn1"]
for dec_col in decimal_list:
    df_PrSlDl[dec_col] = df_PrSlDl[dec_col].str.replace(',', '')
    df_PrSlDl[dec_col] = df_PrSlDl[dec_col].str.replace('(', '-')
    df_PrSlDl[dec_col] = df_PrSlDl[dec_col].str.replace(')', '')
    df_PrSlDl[dec_col] = df_PrSlDl[dec_col].replace('-', '0.00')
    df_PrSlDl[dec_col] = df_PrSlDl[dec_col].astype(float)
    # df_PrSlDl[dec_col] = df_PrSlDl[dec_col].apply(lambda x: '{:,.2f}'.format(x))

df_PrSlDl.head(3)

C:\Users\vuongns\AppData\Local\Temp\ipykernel_14428\3787169930.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_PrSlDl[dec_col] = df_PrSlDl[dec_col].str.replace('(', '-')
C:\Users\vuongns\AppData\Local\Temp\ipykernel_14428\3787169930.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_PrSlDl[dec_col] = df_PrSlDl[dec_col].str.replace(')', '')


,Textbox72,Contact,FirmName,ItmsGrpCod,IndName,CustomerCode,CustomerName,AddressC,CustomerGroup,Province,...,Other,OtherReceipt,Dscription,Amount,AmountReturn,Discount,DiscountAmount,CashDiscount,Amount1,AmountReturn1
0,1,CHI LIEN,ABBOTT,101,ETC,CC002804,NHA THUOC TAY HUONG LIEN,"6 NGUYEN KHAC VIEN, PHUONG TRUONG THI, THANH P...",ACH,THANH HÓA,...,-,-,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,CHI LIEN,ABBOTT,101,ETC,CC002804,NHA THUOC TAY HUONG LIEN,"6 NGUYEN KHAC VIEN, PHUONG TRUONG THI, THANH P...",ACH,THANH HÓA,...,-,-,NaN,490.0,0.0,2.0,1719576.6,0.0,84259253.4,0.0
2,3,CHI YEN,ABBOTT,101,ETC,CC004957,NHA THUOC TAY HIEN,"2 LY TU TRONG, PHUONG HA HUY TAP, THANH PHO VI...",ACH,NGHỆ AN,...,-,-,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# handle Bao cao hang ton theo kho GV

In [14]:
import pandas as pd
from datetime import datetime
from datetime import date
import numpy as np
import datetime as dt

# determine the number of rows to be skip
df_re = pd.read_csv(r"D:\PURCHASING\test_NA\04. Báo cáo hàng tồn theo kho (Giá vốn).csv",
                    skiprows=3,usecols=[0])
skip_r = (df_re[df_re['Warehouse'] == 'ItemCode2'].index[0])+5

# read csv file
df_TKGV = pd.read_csv(r"D:\PURCHASING\test_NA\04. Báo cáo hàng tồn theo kho (Giá vốn).csv",
skiprows=skip_r,header=0,encoding='utf-8-sig')

# remove columns from 'ItemCode2' to 'Textbox114'
df_TKGV.drop(df_TKGV.columns[range(0,26)], axis=1, inplace=True)
df_TKGV.fillna(0, inplace=True)

# fix ExpiredDate1 issue, string to datetime format
df_TKGV['ExpiredDate1'] = df_TKGV['ExpiredDate1'].str.replace('/9999','/2099')
df_TKGV['ExpiredDate1'] = df_TKGV['ExpiredDate1'].str.replace('/2999','/2099')
df_TKGV['ExpiredDate1'] = pd.to_datetime(df_TKGV['ExpiredDate1'])

# fix numeric and string issues
decimal_list = ['Quantity1','OK','ItemCost3',]
for dec_col in decimal_list:
    df_TKGV[dec_col] = df_TKGV[dec_col].str.replace(',', '')
    df_TKGV[dec_col] = df_TKGV[dec_col].str.replace('(', '-')
    df_TKGV[dec_col] = df_TKGV[dec_col].str.replace(')', '')
    df_TKGV[dec_col] = df_TKGV[dec_col].replace('-', '0.00')
    df_TKGV[dec_col] = df_TKGV[dec_col].astype(float)
    # df_TKGV[dec_col] = df_TKGV[dec_col].apply(lambda x: '{:,.2f}'.format(x))

# insert month of report
df_TKGV.insert(0,'Rpt_month',dt.datetime(2023,1,31))

# calculate age of inventory
df_TKGV['AgeOfInv'] = ((pd.to_datetime(df_TKGV['ExpiredDate1']) - pd.to_datetime(df_TKGV['Rpt_month']))/ np.timedelta64(1, 'M')+1).astype(int)

df_TKGV['Status'] = np.where(df_TKGV['AgeOfInv'] > 3,'OK','EXP')

df_TKGV.head(3)

C:\Users\vuongns\AppData\Local\Temp\ipykernel_14428\2357479517.py:23: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_TKGV['ExpiredDate1'] = pd.to_datetime(df_TKGV['ExpiredDate1'])
C:\Users\vuongns\AppData\Local\Temp\ipykernel_14428\2357479517.py:29: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_TKGV[dec_col] = df_TKGV[dec_col].str.replace('(', '-')
C:\Users\vuongns\AppData\Local\Temp\ipykernel_14428\2357479517.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_TKGV[dec_col] = df_TKGV[dec_col].str.replace(')', '')


,Rpt_month,ItemCode1,ItemName1,UoM1,BatchNo1,ExpiredDate1,Warehouse8,Quantity1,OK,HOLD,ItemCost3,CardCode1,Diachi1,AgeOfInv,Status
0,2023-01-31,3M118,118 ATTEST BIOLOG INCUBATOR STEAM 56C,Cái,322411,2099-12-31,WL001,1.0,1.0,0.0,6632547.0,0,0,923,OK
1,2023-01-31,3M1522,1522 BANG KEO 3.6 IN x 60 YD,Cuộn,195,2099-12-31,WL001,17.0,17.0,0.0,700000.0,0,0,923,OK
2,2023-01-31,3M2306,2306 ONG NGHE LITTMANN XANH BIEN,Cái,2015-02AF,2099-12-31,WL001,50.0,50.0,0.0,1585850.0,0,0,923,OK


# handle BC ton kho SAP INFO

## read BC ton kho SAP INFO

In [3]:
import pandas as pd
import numpy as np

# determine the number of rows to be skip
df_re = pd.read_csv(r"D:\PURCHASING\test_NA\04. Báo cáo hàng tồn theo kho SAP - INFOR.csv",
                    skiprows=3,usecols=[0])
skip_r = (df_re[df_re['Warehouse'] == 'Warehouse1'].index[0])+5

# read csv file
df_TKSPIF = pd.read_csv(r"D:\PURCHASING\test_NA\04. Báo cáo hàng tồn theo kho SAP - INFOR.csv",
skiprows=skip_r,header=0,encoding='utf-8-sig')

# fix ExpiredDate issue, string to datetime format
df_TKSPIF['ExpiredDate'] = df_TKSPIF['ExpiredDate'].str.replace('/9999','/2099')
df_TKSPIF['ExpiredDate'] = df_TKSPIF['ExpiredDate'].str.replace('/2999','/2099')
df_TKSPIF['ExpiredDate'] = pd.to_datetime(df_TKSPIF['ExpiredDate'])

# fix numeric and string issues
decimal_list = ["SapQty","QTY"]
for dec_col in decimal_list:
    df_TKSPIF[dec_col] = df_TKSPIF[dec_col].str.replace(',', '')
    df_TKSPIF[dec_col] = df_TKSPIF[dec_col].str.replace('(', '-')
    df_TKSPIF[dec_col] = df_TKSPIF[dec_col].str.replace(')', '')
    df_TKSPIF[dec_col] = df_TKSPIF[dec_col].replace('-', '0.00')
    df_TKSPIF[dec_col] = df_TKSPIF[dec_col].astype(float)
    # df_TKSPIF[dec_col] = df_TKSPIF[dec_col].apply(lambda x: '{:,.2f}'.format(x))

df_TKSPIF.head(3)

C:\Users\vuongns\AppData\Local\Temp\ipykernel_14428\112042588.py:16: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_TKSPIF['ExpiredDate'] = pd.to_datetime(df_TKSPIF['ExpiredDate'])
C:\Users\vuongns\AppData\Local\Temp\ipykernel_14428\112042588.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_TKSPIF[dec_col] = df_TKSPIF[dec_col].str.replace('(', '-')
C:\Users\vuongns\AppData\Local\Temp\ipykernel_14428\112042588.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_TKSPIF[dec_col] = df_TKSPIF[dec_col].str.replace(')', '')


,Warehouse1,ItemCode,ItemName,UoM,ItemGroup1,ItemGroupName,ManufactureName,BatchNo,ExpiredDate,SapQty,STATUS1,PUTAWAYZONE,DESCR,QTY
0,HL001,AB1100001,DUPHALAC 10G/15ML SYR 15ML 20,Hộp,101,DƯỢC PHẨM,M00003 - ABBOTT,366425,2023-09-30,1542.0,OK,ASPICK,KV NHẶT HÀNG LẺ,43.0
1,HL001,AB1100001,DUPHALAC 10G/15ML SYR 15ML 20,Hộp,101,DƯỢC PHẨM,M00003 - ABBOTT,366425,2023-09-30,1542.0,OK,ABSTOR,KV BẢO QUẢN,130.0
2,HL001,AB1100001,DUPHALAC 10G/15ML SYR 15ML 20,Hộp,101,DƯỢC PHẨM,M00003 - ABBOTT,366425,2023-09-30,1542.0,OK,DOCK,Receiving and Shipping Dock,422.0


## filter damage items

In [4]:
df_fil_SPIF = df_TKSPIF.copy()
# df_dmg = df_fil_SPIF[(df_fil_SPIF["DESCR"]
#                       .isin(['KV HÀNG LẠNH CÁCH LY','KV HÀNG CÁCH LY','KV HÀNG LẠNH BỊ LOẠI']))]
df_dmg = df_fil_SPIF[df_fil_SPIF['STATUS1']=='HOLD']
df_dmg.head(3)

,Warehouse1,ItemCode,ItemName,UoM,ItemGroup1,ItemGroupName,ManufactureName,BatchNo,ExpiredDate,SapQty,STATUS1,PUTAWAYZONE,DESCR,QTY
60,HL001,GZ7300001,ILON® INTENSIVE BALM 75ML,Tuýp,107,MỸ PHẨM,M00017 - GETZ,0010,2024-04-30,4530.0,HOLD,AQUARR,KV HÀNG CÁCH LY,628.0
63,HL001,GZ7300001,ILON® INTENSIVE BALM 75ML,Tuýp,107,MỸ PHẨM,M00017 - GETZ,0333,2025-04-30,11451.0,HOLD,AQUARR,KV HÀNG CÁCH LY,320.0
65,HL001,GZ7300002,ILON® ANTI-CALLUS CREAM 75ML,Tuýp,107,MỸ PHẨM,M00017 - GETZ,0011,2024-03-31,3130.0,HOLD,AQUARR,KV HÀNG CÁCH LY,40.0


# Combine data

In [41]:
df_TKGV.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['Rpt_month', 'ItemCode1', 'ItemName1', 'UoM1', 'BatchNo1',
       'ExpiredDate1', 'Warehouse8', 'Quantity1', 'OK', 'HOLD', 'ItemCost3',
       'CardCode1', 'Diachi1', 'AgeOfInv'],
      dtype='object')>

## calculate average COGS

In [22]:
# using pandasql
from pandasql import sqldf
mysql = lambda q: sqldf(q, globals())

qr_av_cogs = """
SELECT DISTINCT ItemCode1,ItemName1,
AVG(ItemCost3) AS Avg_Cogs
FROM df_TKGV
GROUP BY ItemCode1,ItemName1
ORDER BY ItemCode1,ItemName1
"""
df_av_Cogs = mysql(qr_av_cogs)
df_av_Cogs

,ItemCode1,ItemName1,Avg_Cogs
0,3M118,118 ATTEST BIOLOG INCUBATOR STEAM 56C,6632547.00
1,3M1522,1522 BANG KEO 3.6 IN x 60 YD,700000.00
2,3M2306,2306 ONG NGHE LITTMANN XANH BIEN,1585850.00
3,3M260-032,260-032 DAY DUY TRI 0.32 7'',408569.00
4,3M290G,290G MAY 240 VAC DOC MAU TEST DO DO TIET TRUNG...,41352109.94
...,...,...,...
1584,YH6100009,THUC PHAM BAO VE SUC KHOE YES SEAWEED CALCIUM ...,545455.00
1585,YH6100010,THUC PHAM BAO VE SUC KHOE YES NUTRI COMPLEX,263636.00
1586,YH6100011,THUC PHAM BAO VE SUC KHOE YES VIMICAN,363636.00
1587,YH6100012,THUC PHAM BAO VE SUC KHOE YES KIDDO SEACAL+D,368182.00


## all item

In [17]:
df_PrSlDl.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['Textbox72', 'Contact', 'FirmName', 'ItmsGrpCod', 'IndName',
       'CustomerCode', 'CustomerName', 'AddressC', 'CustomerGroup', 'Province',
       'District', 'SDate', 'DocumentDate', 'SlpName', 'SOno', 'ProductID',
       'ProductName', 'Unit', 'Price', 'BasePrice', 'Variant', 'Qty',
       'SaleReturn', 'Bonus', 'BonusReturn', 'Saple', 'SampleReturn',
       'Consignment', 'ConsignmentReturn', 'Other', 'OtherReceipt',
       'Dscription', 'Amount', 'AmountReturn', 'Discount', 'DiscountAmount',
       'CashDiscount', 'Amount1', 'AmountReturn1'],
      dtype='object')>

In [18]:
df_dmg.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['Warehouse1', 'ItemCode', 'ItemName', 'UoM', 'ItemGroup1',
       'ItemGroupName', 'ManufactureName', 'BatchNo', 'ExpiredDate', 'SapQty',
       'STATUS1', 'PUTAWAYZONE', 'DESCR', 'QTY'],
      dtype='object')>

In [19]:
df_TKGV.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['Rpt_month', 'ItemCode1', 'ItemName1', 'UoM1', 'BatchNo1',
       'ExpiredDate1', 'Warehouse8', 'Quantity1', 'OK', 'HOLD', 'ItemCost3',
       'CardCode1', 'Diachi1', 'AgeOfInv', 'Status'],
      dtype='object')>

In [23]:
df_av_Cogs.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['ItemCode1', 'ItemName1', 'Avg_Cogs'], dtype='object')>

In [30]:
# using pandasql
from pandasql import sqldf
mysql = lambda q: sqldf(q, globals())

qr_fn = """
SELECT DISTINCT PS.FirmName,TV.ItemCode1,TV.ItemName1,
SUM(CASE WHEN strftime('%Y-%m',PS.DocumentDate)=='2023-01' THEN PS.Amount ELSE 0 END) AS [Sale_2023_01],
AVG(CASE WHEN strftime('%Y-%m',TV.Rpt_month)=='2023-01' THEN TV.ItemCost3 ELSE 0 END) AS [Avg_Cogs],
SUM(TV.Quantity1) AS [Ttl_stock_qty]
FROM df_TKGV TV
LEFT JOIN df_PrSlDl PS
ON PS.ProductID = TV.ItemCode1
LEFT JOIN df_dmg DM
ON DM.ItemCode = TV.ItemCode1
WHERE DM.Status = 'OK'
GROUP BY PS.FirmName,TV.ItemCode1,TV.ItemName1
ORDER BY PS.FirmName,TV.ItemCode1,TV.ItemName1

"""

df_fn_rev = mysql(qr_fn)

# format float columns
pd.set_option('display.float_format',lambda x: '{:,.0f}'.format(x))
df_fn_rev.tail(3)

PandaSQLException: (sqlite3.OperationalError) no such column: DM.Status
[SQL: 
SELECT DISTINCT PS.FirmName,TV.ItemCode1,TV.ItemName1,
SUM(CASE WHEN strftime('%Y-%m',PS.DocumentDate)=='2023-01' THEN PS.Amount ELSE 0 END) AS [Sale_2023_01],
AVG(CASE WHEN strftime('%Y-%m',TV.Rpt_month)=='2023-01' THEN TV.ItemCost3 ELSE 0 END) AS [Avg_Cogs],
SUM(TV.Quantity1) AS [Ttl_stock_qty]
FROM df_TKGV TV
LEFT JOIN df_PrSlDl PS
ON PS.ProductID = TV.ItemCode1
LEFT JOIN df_dmg DM
ON DM.ItemCode = TV.ItemCode1
WHERE DM.Status = 'OK'
GROUP BY PS.FirmName,TV.ItemCode1,TV.ItemName1
ORDER BY PS.FirmName,TV.ItemCode1,TV.ItemName1

]
(Background on this error at: https://sqlalche.me/e/14/e3q8)